In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [143]:
dataFrame = pd.read_csv('capture20110815-2.csv')   #Very small capture (use this for speed of testing)
#dataFrame = pd.read_csv('combinedCapture.csv')

In [144]:
dataFrame.head(5)

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/15 16:43:28.078942,0.000000,tcp,114.33.245.44,6881,?>,147.32.84.118,1567,RA_,0.0,0.0,1,60,60,flow=Background
1,2011/08/15 16:43:32.283576,13.431962,tcp,212.93.105.52,49237,->,147.32.84.229,80,SRA_SA,0.0,0.0,6,388,208,flow=Background-TCP-Established
2,2011/08/15 16:43:32.456441,13.350228,tcp,212.93.105.52,14906,->,147.32.84.229,13363,SRA_SA,0.0,0.0,6,388,208,flow=Background-TCP-Established
3,2011/08/15 16:43:32.850648,13.010090,tcp,212.93.105.52,60349,->,147.32.84.229,443,SRA_SA,0.0,0.0,6,388,208,flow=Background-TCP-Established
4,2011/08/15 16:45:09.305002,20.990047,tcp,115.127.24.116,3198,->,147.32.84.229,443,SR_SA,0.0,0.0,5,308,122,flow=Background-TCP-Established


In [145]:
#Deletes row's where the column values are null,nan,nat, or blank
def deleteNullRow(dataFrame, column):
    newDataFrame = dataFrame
    
    #dataframe dropna won't replace empty values only NaN and NaT so convert blank space to NaN then drop
    newDataFrame[column].replace('', np.nan, inplace=True)
    newDataFrame = newDataFrame.dropna(subset=[column])
    return newDataFrame

In [146]:
# From: https://github.com/mgarzon/cybersec/blob/master/MalwareDetection.ipynb
def preprocessData(dataFrame):

    '''
    This function is used to perform
    the necessary operations to 
    convert the raw data into a
    clean data set.
    '''
    
    #Outputting number of rows and column names before preprocessing
    print("----------Before pre-processing-----------")
    print("Number of rows: " + str(len(dataFrame.index)))
    print("The columns are: " + str(list(dataFrame)))
    
    
    
    #dropping columns specified
    listOfFeaturesToDrop = [
    'Dir',
    'sTos',
    'dTos']
    dataFrame = dataFrame.drop(listOfFeaturesToDrop, axis=1)

    #Dropping all null value rows from specified columns
    dataFrame = deleteNullRow(dataFrame,'Sport')
    dataFrame = deleteNullRow(dataFrame,'SrcAddr')
    dataFrame = deleteNullRow(dataFrame,'Dport')
    dataFrame = deleteNullRow(dataFrame,'DstAddr')
    
    
    # TODO
    #dp.convertColumnToTimeStamp(dataFrame,'StartTime') # ?? already a timestamp
    
    
    #Outputting number of rows and column names after preprocessing
    print("\n----------After pre-processing-----------")
    print("Number of rows: " + str(len(dataFrame.index)))
    print("The columns are: " + str(list(dataFrame)))
    return dataFrame

In [147]:
dataFrame = preprocessData(dataFrame)

----------Before pre-processing-----------
Number of rows: 129832
The columns are: ['StartTime', 'Dur', 'Proto', 'SrcAddr', 'Sport', 'Dir', 'DstAddr', 'Dport', 'State', 'sTos', 'dTos', 'TotPkts', 'TotBytes', 'SrcBytes', 'Label']

----------After pre-processing-----------
Number of rows: 128882
The columns are: ['StartTime', 'Dur', 'Proto', 'SrcAddr', 'Sport', 'DstAddr', 'Dport', 'State', 'TotPkts', 'TotBytes', 'SrcBytes', 'Label']


In [148]:
dataFrame.head(5)

,StartTime,Dur,Proto,SrcAddr,Sport,DstAddr,Dport,State,TotPkts,TotBytes,SrcBytes,Label
0,2011/08/15 16:43:28.078942,0.000000,tcp,114.33.245.44,6881,147.32.84.118,1567,RA_,1,60,60,flow=Background
1,2011/08/15 16:43:32.283576,13.431962,tcp,212.93.105.52,49237,147.32.84.229,80,SRA_SA,6,388,208,flow=Background-TCP-Established
2,2011/08/15 16:43:32.456441,13.350228,tcp,212.93.105.52,14906,147.32.84.229,13363,SRA_SA,6,388,208,flow=Background-TCP-Established
3,2011/08/15 16:43:32.850648,13.010090,tcp,212.93.105.52,60349,147.32.84.229,443,SRA_SA,6,388,208,flow=Background-TCP-Established
4,2011/08/15 16:45:09.305002,20.990047,tcp,115.127.24.116,3198,147.32.84.229,443,SR_SA,5,308,122,flow=Background-TCP-Established


In [149]:
from sklearn import preprocessing 

#Function to perform discretization on the data
def discretizeData(dataFrame):
    
    dfNew = dataFrame
    
    # Binning technique from
    # https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b
    quantile_list = [0, .25, .5, .75, 1.] # Change the quantile_list for more or less accuracy
    
    dfNew['TotBytesDisc'] = ""
    dfNew['SrcBytesDisc'] = ""
    dfNew['TotBytesDisc'] = pd.qcut(dataFrame['TotBytes'], quantile_list)
    dfNew['SrcBytesDisc'] = pd.qcut(dataFrame['SrcBytes'], quantile_list)
    
    # Bin Src/Dest port
    # According to 0-1023(WELLKNOWN_PORTNUMBER)
    #              1024-49151(REGISTERED_PORTNUMBER)
    #              49152-65535(DYNAMIC_PORTNUMBER)
    Sport = dataFrame['Sport']#[0x0303].astype('int64')
    Sport = Sport.apply(lambda x: int(x, 16) if x[0] == '0' and x[1] == 'x' else int(x, 10)) # TODO, there has to be better way
    dfNew['SportDisc'] = ""
    dfNew['SportDisc'] = pd.cut(Sport, [0, 1023, 49151, 65535])
    
    Dport = dataFrame['Dport']#[0x0303].astype('int64')
    Dport = Dport.apply(lambda x: int(x, 16) if x[0] == '0' and x[1] == 'x' else int(x, 10))
    dfNew['DportDisc'] = ""
    dfNew['DportDisc'] = pd.cut(Dport, [0, 1023, 49151, 65535])

    
    #LabelEncoder for unique values for Proto column and stored as column ProtoDisc
    le = preprocessing.LabelEncoder()
    le.fit(dfNew.Proto.unique())
    dfNew["ProtoDisc"] = ""
    dfNew.ProtoDisc = le.transform(dfNew.Proto)
    
    
    #Encoding "label" column to "labelDisc"
    #0 = Background/Normal             1=Botnet
    dfNew["LabelDisc"] = ""
    dfNew['LabelDisc'] = dfNew['Label']
    dfNew['LabelDisc'] = dfNew.LabelDisc.str.replace(r'(^.*Background.*$)', '0')
    dfNew['LabelDisc'] = dfNew.LabelDisc.str.replace(r'(^.*Normal.*$)', '0')
    dfNew['LabelDisc'] = dfNew.LabelDisc.str.replace(r'(^.*Botnet.*$)', '1')
    
    
    return dfNew
    


In [150]:
dataFrame = discretizeData(dataFrame)

In [151]:
dataFrame.head(5)

,StartTime,Dur,Proto,SrcAddr,Sport,DstAddr,Dport,State,TotPkts,TotBytes,SrcBytes,Label,TotBytesDisc,SrcBytesDisc,SportDisc,DportDisc,ProtoDisc,LabelDisc
0,2011/08/15 16:43:28.078942,0.000000,tcp,114.33.245.44,6881,147.32.84.118,1567,RA_,1,60,60,flow=Background,"(59.999, 214.0]","(-0.001, 77.0]","(1023, 49151]","(1023, 49151]",3,0
1,2011/08/15 16:43:32.283576,13.431962,tcp,212.93.105.52,49237,147.32.84.229,80,SRA_SA,6,388,208,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(49151, 65535]","(0, 1023]",3,0
2,2011/08/15 16:43:32.456441,13.350228,tcp,212.93.105.52,14906,147.32.84.229,13363,SRA_SA,6,388,208,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(1023, 49151]","(1023, 49151]",3,0
3,2011/08/15 16:43:32.850648,13.010090,tcp,212.93.105.52,60349,147.32.84.229,443,SRA_SA,6,388,208,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(49151, 65535]","(0, 1023]",3,0
4,2011/08/15 16:45:09.305002,20.990047,tcp,115.127.24.116,3198,147.32.84.229,443,SR_SA,5,308,122,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(1023, 49151]","(0, 1023]",3,0


In [152]:
def generateSrcAddrFeaturesConnectionBased(dataFrame, windowSize):
    
    dfNew = dataFrame
    
    le = preprocessing.LabelEncoder()
    le.fit(dfNew.SrcAddr.unique())
    dfNew["SrcAddr_Dis"] = ""
    dfNew["SrcAddr_Dis"] = le.transform(dfNew.SrcAddr)
    
    #How many times the SRCADDRESS has appeared within the last X netflows
    def countMatching(x):
        num = 0
        for i in x:
            num += 1 if i==x[-1] else 0
        return num
    dfNew['Nbr_SRCAddr_Appeared'] = dataFrame['SrcAddr_Dis'].rolling(windowSize).apply(countMatching)

    
    return dfNew
    

In [154]:
dataFrameTest = dataFrame
#Window size 10 for testing, actual use 10,000
dataFrameTest = generateSrcAddrFeaturesConnectionBased(dataFrame,10)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  app.launch_new_instance()


In [155]:
dataFrameTest.head(50)

,StartTime,Dur,Proto,SrcAddr,Sport,DstAddr,Dport,State,TotPkts,TotBytes,SrcBytes,Label,TotBytesDisc,SrcBytesDisc,SportDisc,DportDisc,ProtoDisc,LabelDisc,SrcAddr_Dis,Nbr_SRCAddr_Appeared
0,2011/08/15 16:43:28.078942,0.000000,tcp,114.33.245.44,6881,147.32.84.118,1567,RA_,1,60,60,flow=Background,"(59.999, 214.0]","(-0.001, 77.0]","(1023, 49151]","(1023, 49151]",3,0,1798,NaN
1,2011/08/15 16:43:32.283576,13.431962,tcp,212.93.105.52,49237,147.32.84.229,80,SRA_SA,6,388,208,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(49151, 65535]","(0, 1023]",3,0,10612,NaN
2,2011/08/15 16:43:32.456441,13.350228,tcp,212.93.105.52,14906,147.32.84.229,13363,SRA_SA,6,388,208,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(1023, 49151]","(1023, 49151]",3,0,10612,NaN
3,2011/08/15 16:43:32.850648,13.010090,tcp,212.93.105.52,60349,147.32.84.229,443,SRA_SA,6,388,208,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(49151, 65535]","(0, 1023]",3,0,10612,NaN
4,2011/08/15 16:45:09.305002,20.990047,tcp,115.127.24.116,3198,147.32.84.229,443,SR_SA,5,308,122,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(1023, 49151]","(0, 1023]",3,0,1961,NaN
5,2011/08/15 16:45:27.991372,12.542819,tcp,115.127.24.116,3196,147.32.84.229,13363,SR_SA,5,308,122,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(1023, 49151]","(1023, 49151]",3,0,1961,NaN
6,2011/08/15 16:45:30.295050,13.308726,tcp,115.127.24.116,3198,147.32.84.229,443,SR_SA,5,308,122,flow=Background-TCP-Established,"(268.0, 757.0]","(81.0, 463.0]","(1023, 49151]","(0, 1023]",3,0,1961,NaN
7,2011/08/15 16:45:57.756664,1.413248,tcp,77.52.60.161,3767,147.32.84.118,6881,S_RA,4,244,124,flow=Background-TCP-Attempt,"(214.0, 268.0]","(81.0, 463.0]","(1023, 49151]","(1023, 49151]",3,0,17267,NaN
8,2011/08/15 16:47:50.502720,5.147244,tcp,77.52.60.161,3823,147.32.84.118,6881,S_RA,4,244,124,flow=Background-TCP-Attempt,"(214.0, 268.0]","(81.0, 463.0]","(1023, 49151]","(1023, 49151]",3,0,17267,NaN
9,2011/08/15 16:47:53.430662,3.001157,tcp,147.32.84.59,52956,77.75.72.72,80,SR_A,3,186,126,flow=Background-Established-cmpgw-CVUT,"(59.999, 214.0]","(81.0, 463.0]","(49151, 65535]","(0, 1023]",3,0,4343,1.0
